# Clasificación de Imágenes MRI con SVM + HOG

Este notebook implementa un pipeline clásico de visión computacional usando:

1. Extracción de características HOG (Histogram of Oriented Gradients)
2. Clasificación con SVM (Support Vector Machine)
3. Evaluación de resultados

In [ ]:
import os
import numpy as np
from pathlib import Path
import cv2
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

BASE_DIR = Path("../total/archive/Training")
classes = [d.name for d in BASE_DIR.iterdir() if d.is_dir()]
num_classes = len(classes)
IMG_SIZE = 128

X, y = [], []
for idx, class_name in enumerate(classes):
    class_dir = BASE_DIR / class_name
    for img_path in class_dir.glob('*.jpg'):
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
        X.append(features)
        y.append(idx)
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

clf = SVC(kernel='linear', probability=True, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=classes))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title('Matriz de Confusión')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos')
plt.show()

Este pipeline es útil como línea base y para comparar con modelos de deep learning.